# DeepLizard PyTorch Videos

In [134]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [43]:
# Video 1
# Torch tensor multiplication
a = torch.tensor([4,12,5,4],dtype = torch.float32)
print(a,a.shape)
b = torch.randint(0,20,size=(5,4),dtype=torch.float32)
print(b)
b.matmul(a)

tensor([ 4., 12.,  5.,  4.]) torch.Size([4])
tensor([[ 3., 11., 10.,  8.],
        [14.,  8.,  8.,  1.],
        [ 5.,  9.,  3.,  7.],
        [15., 15.,  5.,  8.],
        [ 8., 13., 17.,  2.]])


tensor([226., 196., 171., 297., 281.])

In [46]:
fc = nn.Linear(4,5,bias = False)
fc(a) # passing the input to the hiidden la
fc.weight # these are the random assigned weights

Parameter containing:
tensor([[ 0.2378, -0.3530, -0.0535, -0.0147],
        [-0.1691, -0.3778,  0.2041, -0.0449],
        [ 0.2840,  0.4596, -0.2715,  0.4295],
        [ 0.1316, -0.1160, -0.1368,  0.2888],
        [ 0.3611, -0.3528,  0.2873, -0.1007]], requires_grad=True)

In [47]:
# setting the weight to the predefined one
fc.weight = nn.Parameter(b)
fc(a)

tensor([226., 196., 171., 297., 281.], grad_fn=<SqueezeBackward3>)

In [53]:
# PRin number of elements within the tensors
print(b.numel())
torch.tensor(b.shape).prod()

20


tensor(20)

In [58]:
# implementing flatten function
def flatten(t):
    t = t.reshape(-1)
    return t
print(flatten(b))

tensor([ 3., 11., 10.,  8., 14.,  8.,  8.,  1.,  5.,  9.,  3.,  7., 15., 15.,
         5.,  8.,  8., 13., 17.,  2.])


In [66]:
# concat tensors into different dimenstions
t1 = torch.tensor([[1,2],[3,4]],dtype=torch.float32)
t2 = torch.tensor([[5,6],[7,8]],dtype=torch.float32)
t3 = torch.tensor([[9,10],[11,12]],dtype=torch.float32)
# dim 0 concat
print(torch.cat((t1,t2)))
# dim 1 concat
print(torch.cat((t1,t2),dim=1))
# stack is used to merge tensors channel wise
print(torch.stack((t1,t2,t3)))  # <----- imp


tensor([[1., 2.],
        [3., 4.],
        [5., 6.],
        [7., 8.]])
tensor([[1., 2., 5., 6.],
        [3., 4., 7., 8.]])
tensor([[[ 1.,  2.],
         [ 3.,  4.]],

        [[ 5.,  6.],
         [ 7.,  8.]],

        [[ 9., 10.],
         [11., 12.]]])


# Creating a dummy model
* adding 2 conv layer 
* adding 2 dense layer
* implementing forward pass operation
* generate a fake tensor and pass through the network

In [87]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=5,kernel_size=5)
        self.conv2 = nn.Conv2d(5,4,kernel_size=3,stride =3) #(8,8) output
        self.fc1 = nn.Linear(4*4*4,10)
        self.out = nn.Linear(10,5)
    
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.reshape(-1)
        x = F.relu(self.fc1(x))
        return F.relu(self.out(x))

    
torch.set_grad_enabled(False)  # stop the computational graph calculation

    
net = Network() # Model initialized
print(net.parameters)   # display the parameters of the model


# generate demo image
image = torch.randint(0,255,(1,28,28),dtype = torch.float32).unsqueeze(dim=0)
print(image.shape)

# pass the image from the network
net(image)

<bound method Module.parameters of Network(
  (conv1): Conv2d(1, 5, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(5, 4, kernel_size=(3, 3), stride=(3, 3))
  (fc1): Linear(in_features=64, out_features=10, bias=True)
  (out): Linear(in_features=10, out_features=5, bias=True)
)>
torch.Size([1, 1, 28, 28])


tensor([0.0000, 4.1054, 0.7554, 6.2099, 2.5689])

# Deep NN Model to predict the function ouput
## Make NN with dense structure
- 3 linear layer
- generater data generating fucntion and load it with data loader
* iterate and run epochs 
* plot the accuracy and loss curves`

In [204]:
# Defining the Liner model 
from torch.utils.data import Dataset, DataLoader
class FuncApproximator(nn.Module):
    def __init__(self,input_features,output_features):
        super().__init__()
        self.fc1 = nn.Linear(input_features,5)
        self.out = nn.Linear(5,output_features)
    
    def forward(self,x):
        x = self.fc1(x)
        return self.out(x)
torch.set_grad_enabled(True)
net = FuncApproximator(2,1)
net

FuncApproximator(
  (fc1): Linear(in_features=2, out_features=5, bias=True)
  (out): Linear(in_features=5, out_features=1, bias=True)
)

In [210]:
# Dataset generation
def gen_dataset(samples):
    a = torch.randn(size =(samples,2),dtype = torch.float32)
    # the function defined
    
#     print(a.shape)
    c = 2*a[:,0]+5*a[:,1]
#     print(c)
#     a = a.unsqueeze(dim=0)
#     print(a.shape, c.shape)
#     d = torch.from_numpy(np.array([a.numpy(),c.numpy()]))
#     print(d)
#     print(d.shape)
    return (a,c) #torch.cat( (a.unsqueeze(0).t(),c),dim=0)
tx,ty = gen_dataset(1000)

In [211]:
optimizer = torch.optim.Adam(net.parameters(),lr = 1)

In [212]:
ty = ty.reshape(-1,1)

In [213]:
lossM = nn.MSELoss()

In [214]:
# main loop
epochs = 15
for epoch_ in range(epochs):
    total_loss = 0
    for i,j in zip(tx,ty):
        optimizer.zero_grad()
        pred = net(i)
        loss = lossM(pred,j)
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    print(f"epochs: {epoch_} .. loss : {total_loss} ")


epochs: 0 .. loss : 63714.04568270729 
epochs: 1 .. loss : 1126248.7258909005 
epochs: 2 .. loss : 57.80890582227856 
epochs: 3 .. loss : 244006.9051575576 
epochs: 4 .. loss : 9.442334830689347e-10 
epochs: 5 .. loss : 1.0381388110269896e-09 
epochs: 6 .. loss : 1.14101442877379e-08 
epochs: 7 .. loss : 1449.0028505587711 
epochs: 8 .. loss : 293.66410333112447 
epochs: 9 .. loss : 3455282.8287907606 
epochs: 10 .. loss : 1.561834074570445e-08 
epochs: 11 .. loss : 3.593939715273997e-09 
epochs: 12 .. loss : 0.0013838356388997664 
epochs: 13 .. loss : 9616.114167461243 
epochs: 14 .. loss : 1.109148723005271e-07 
